# Local Accounts Discovery
Local accounts, System Owner/User discovery using operating systems utilities

## Rule Content
```
- title: Local Accounts Discovery
  id: 502b42de-4306-40b4-9596-6f590c81f073
  status: experimental
  description: Local accounts, System Owner/User discovery using operating systems
    utilities
  author: Timur Zinniatullin, Daniil Yugoslavskiy, oscd.community
  date: 2019/10/21
  modified: 2019/11/04
  references:
  - https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1033/T1033.yaml
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection_1:
    - Image|endswith: \whoami.exe
    - Image|endswith: \wmic.exe
      CommandLine|contains|all:
      - useraccount
      - get
    - Image|endswith:
      - \quser.exe
      - \qwinsta.exe
    - Image|endswith: \cmdkey.exe
      CommandLine|contains: /list
    - Image|endswith: \cmd.exe
      CommandLine|contains|all:
      - /c
      - dir
      - \Users\
    selection_2:
      Image|endswith:
      - \net.exe
      - \net1.exe
      CommandLine|contains: user
    filter:
      CommandLine|contains:
      - /domain
      - /add
      - /delete
      - /active
      - /expires
      - /passwordreq
      - /scriptpath
      - /times
      - /workstations
    condition: selection_1 or ( selection_2 and not filter )
  fields:
  - Image
  - CommandLine
  - User
  - LogonGuid
  - Hashes
  - ParentProcessGuid
  - ParentCommandLine
  falsepositives:
  - Legitimate administrator or user enumerates local users for legitimate reason
  level: low
  tags:
  - attack.discovery
  - attack.t1033
  - attack.t1087

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:*\\whoami.exe OR (process_path.keyword:*\\wmic.exe AND process_command_line.keyword:*useraccount* AND process_command_line.keyword:*get*) OR process_path.keyword:(*\\quser.exe OR *\\qwinsta.exe) OR (process_path.keyword:*\\cmdkey.exe AND process_command_line.keyword:*\/list*) OR (process_path.keyword:*\\cmd.exe AND process_command_line.keyword:*\/c* AND process_command_line.keyword:*dir* AND process_command_line.keyword:*\\Users\*)) OR ((process_path.keyword:(*\\net.exe OR *\\net1.exe) AND process_command_line.keyword:*user*) AND (NOT (process_command_line.keyword:(*\/domain* OR *\/add* OR *\/delete* OR *\/active* OR *\/expires* OR *\/passwordreq* OR *\/scriptpath* OR *\/times* OR *\/workstations*)))))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()